https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_multiclass_classification.ipynb

In [1]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
device

'cuda'

## formating data

In [4]:
def encode_cat(string, ref):
     return ref.index(string)

In [5]:
# Import data from csv
df = pd.read_csv('../datasets/cleaned/combined_text.csv')

# drop duplicates
df.drop_duplicates(subset=['combined_text'], keep='first', ignore_index=True, inplace=True)

# encode category
category_list = list(df.category.unique())
df['label'] = df['category'].map(lambda x: encode_cat(x, category_list))

df.drop(columns=['asin'], inplace=True)
df.rename(columns={'combined_text': 'text'}, inplace=True)

In [6]:
len(category_list)

21

In [7]:
df.head(2)

,text,category,label
0,I have a 9 year old Badger 1 that needs replac...,appliances,0
1,model number This may help InSinkErator Model ...,appliances,0


## Preparing dataset and dataloader

In [8]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')


In [9]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        text = str(self.data.text[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.label[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [10]:
# Creating the dataset and dataloader for the neural network

train_size = 0.9
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (1271009, 3)
TRAIN Dataset: (1143908, 3)
TEST Dataset: (127101, 3)


In [11]:
train_dataset

,text,category,label
0,Does this have a 240V Twist-lock socket? It lo...,patio_lawn_and_garden,14
1,i don't see a plug underneath the unit where t...,home_and_kitchen,10
2,Will these fit an Artic Cat Wilcat 2014 as lon...,automotive,2
3,I will have no problem when I get the phone tu...,cell_phones_and_accessories,5
4,jWhat is the size of each stamp? I need someth...,arts_crafts_and_sewing,1
...,...,...,...
1143903,"On this product description, it doesn't say if...",beauty,4
1143904,What are the actual measurements of this item?...,tools_and_home_improvement,18
1143905,can i use this in my 2014 Forrester (non turbo...,automotive,2
1143906,"DRM? Hi, Does anyone know what DRM this compil...",video_games,20


In [12]:
test_dataset

,text,category,label
0,can I replace Badger 1 1/3 with a Badger 5 1/2...,appliances,0
1,Can someone tell me if this Badger 5 disposer ...,appliances,0
2,My present disposal has a 3 bolt connection. W...,appliances,0
3,can you bake corn bread on this without it bur...,appliances,0
4,"I need a device to let my dryer vent out, I li...",appliances,0
...,...,...,...
127096,How do I get this to work for two players on p...,video_games,20
127097,how many gig on the hard drive It is a 500gb h...,video_games,20
127098,Does the kinect-less version still play blu ra...,video_games,20
127099,Are there offline options for the games for Xb...,video_games,20


In [13]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

## Creating the Neural Network for Fine Tuning

In [14]:
# Creating the customized model, 
# by adding a drop out and a dense layer on top of distil bert 
# to get the final output for the model. 

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
#         self.classifier = torch.nn.Linear(768, 4)
        self.classifier = torch.nn.Linear(768, 21)
#         self.classifier = torch.nn.Linear(768, 20)
    
        

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [15]:
model = DistillBERTClass()
model.to(device)

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

In [16]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

## Fine Tuning the Model

In [17]:
# Function to calcuate the accuracy of the model

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [37]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch, loader = training_loader):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [19]:
%%time
for epoch in range(EPOCHS):
    train(epoch)

C:\Users\mriva\.conda\envs\torch\lib\site-packages\transformers\tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 5000 steps: 2.9777770042419434
Training Accuracy per 5000 steps: 25.0
Training Loss per 5000 steps: 2.4404312234131296
Training Accuracy per 5000 steps: 28.799240151969606
Training Loss per 5000 steps: 2.314353849953764
Training Accuracy per 5000 steps: 32.4042595740426
Training Loss per 5000 steps: 2.220172642506239
Training Accuracy per 5000 steps: 35.169322045196985
Training Loss per 5000 steps: 2.134727981673331
Training Accuracy per 5000 steps: 37.68811559422029
Training Loss per 5000 steps: 2.0662048201359684
Training Accuracy per 5000 steps: 39.72741090356386
Training Loss per 5000 steps: 2.011505581018094
Training Accuracy per 5000 steps: 41.392786907103094
Training Loss per 5000 steps: 1.966009432741938
Training Accuracy per 5000 steps: 42.70806548384332
Training Loss per 5000 steps: 1.9250804053255879
Training Accuracy per 5000 steps: 43.91077723056924
Training Loss per 5000 steps: 1.8892987684284297
Training Accuracy per 5000 steps: 44.92622386169196
Traini

## validating model

In [34]:
def valid(model, testing_loader):
    tr_loss=0
    nb_tr_steps=0
    nb_tr_examples=0
    
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            
#             outputs = model(ids, mask).squeeze()
            outputs = model(ids, mask)
            
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [35]:
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, testing_loader)
print(f"Accuracy on test data = {acc}")

This is the validation section to print the accuracy and see how it performs
Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch
Validation Loss per 100 steps: 0.9261605739593506
Validation Accuracy per 100 steps: 100.0


C:\Users\mriva\.conda\envs\torch\lib\site-packages\transformers\tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Validation Loss per 100 steps: 1.2189047949886223
Validation Accuracy per 100 steps: 63.17736452709458
Validation Loss per 100 steps: 1.1935676267591655
Validation Accuracy per 100 steps: 63.918608139186084
Validation Loss per 100 steps: 1.1847718260950362
Validation Accuracy per 100 steps: 64.29571361909206
Validation Loss per 100 steps: 1.1859883772218647
Validation Accuracy per 100 steps: 64.36678166091696
Validation Loss per 100 steps: 1.185738560735267
Validation Accuracy per 100 steps: 64.37342506299748
Validation Loss per 100 steps: 1.1825717648910097
Validation Accuracy per 100 steps: 64.48285057164762
Validation Loss per 100 steps: 1.179036021691308
Validation Accuracy per 100 steps: 64.55958401188538
Validation Loss per 100 steps: 1.1784293427084598
Validation Accuracy per 100 steps: 64.5308867278318
Validation Loss per 100 steps: 1.178874219909121
Validation Accuracy per 100 steps: 64.60411990844648
Validation Loss per 100 steps: 1.178964356685378
Validation Accuracy per 100

## Tuning with rest of dataset

In [39]:
%%time
for epoch in range(EPOCHS):
    train(epoch, loader=testing_loader)

C:\Users\mriva\.conda\envs\torch\lib\site-packages\transformers\tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 5000 steps: 0.5841197371482849
Training Accuracy per 5000 steps: 75.0
Training Loss per 5000 steps: 1.1285755676335631
Training Accuracy per 5000 steps: 65.74685062987403
Training Loss per 5000 steps: 1.1267338031292955
Training Accuracy per 5000 steps: 65.91090890910908
Training Loss per 5000 steps: 1.135747613817797
Training Accuracy per 5000 steps: 65.70728618092127
Training Loss per 5000 steps: 1.1326839851631203
Training Accuracy per 5000 steps: 65.88545572721364
Training Loss per 5000 steps: 1.1298973110455015
Training Accuracy per 5000 steps: 65.97736090556377
Training Loss per 5000 steps: 1.1306134047355987
Training Accuracy per 5000 steps: 65.98196726775774
Training Loss per 5000 steps: 1.1323456350619614
Training Accuracy per 5000 steps: 65.95097282934773
Training Loss per 5000 steps: 1.1318313335023007
Training Accuracy per 5000 steps: 65.9564760880978
Training Loss per 5000 steps: 1.1322432220282772
Training Accuracy per 5000 steps: 65.9657563165263
Traini

## Saving the Trained Model Artifacts for inference

In [40]:
# Saving the files for re-use

output_model_file = '../models/pytorch_distilbert_amazon2.bin'
output_vocab_file = '../models/vocab_distilbert_amazon2.bin'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')

All files saved
